In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:40:14.182244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:40:14.291817: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:40:14.291831: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:40:14.313009: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:40:14.849208: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 12:40:19.292123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:40:19.292737: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:40:19.292801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:40:19.292844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:40:19.292885: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 396ms/step - loss: 2.3019 - auc: 0.4019
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7152 - auc: 0.7869
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7920 - auc: 0.8316
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.8161 - auc: 0.8584
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7352 - auc: 0.8694
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6548 - auc: 0.8749
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4916 - auc: 0.8838
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3351 - auc: 0.8934
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2329 - auc: 0.8985
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1789 - auc: 0.9015
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1870 

1/1 [==============================] - 0s 26ms/step - loss: 0.2379 - auc: 0.9905
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2359 - auc: 0.9906
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2339 - auc: 0.9915
Epoch 91/100
1/1 [==============================] - 0s 36ms/step - loss: 0.2377 - auc: 0.9898
Epoch 92/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2320 - auc: 0.9918
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2331 - auc: 0.9919
Epoch 94/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2255 - auc: 0.9928
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2378 - auc: 0.9902
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2378 - auc: 0.9905
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2284 - auc: 0.9912
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Connectivity','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 2.1697 - auc: 0.3962
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5570 - auc: 0.7887
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6231 - auc: 0.8361
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6736 - auc: 0.8489
Epoch 5/100
1/1 [==============================] - 0s 39ms/step - loss: 1.6524 - auc: 0.8574
Epoch 6/100
1/1 [==============================] - 0s 39ms/step - loss: 1.5569 - auc: 0.8568
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4260 - auc: 0.8654
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2811 - auc: 0.8716
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1770 - auc: 0.8780
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1365 - auc: 0.8717
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2657 - auc: 0.9880
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2783 - auc: 0.9851
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2742 - auc: 0.9862
Epoch 91/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2750 - auc: 0.9858
Epoch 92/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2765 - auc: 0.9849
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2736 - auc: 0.9862
Epoch 94/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2806 - auc: 0.9842
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2684 - auc: 0.9871
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2705 - auc: 0.9866
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2643 - auc: 0.9880
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [5]:
m='remove_Connectivity'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 1.7139 - auc: 0.7304
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6037 - auc: 0.7920
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4455 - auc: 0.8074
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3321 - auc: 0.8339
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2255 - auc: 0.8736
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1350 - auc: 0.9066
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.0560 - auc: 0.9265
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9755 - auc: 0.9436
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9265 - auc: 0.9509
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 0.8678 - auc: 0.9620
Epoch 11/100
1/1 [==============================] - 0s 37ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2218 - auc: 0.9923
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2214 - auc: 0.9922
Epoch 90/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2280 - auc: 0.9906
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2263 - auc: 0.9909
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2207 - auc: 0.9919
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2221 - auc: 0.9917
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2276 - auc: 0.9906
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2170 - auc: 0.9932
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2148 - auc: 0.9930
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2279 - auc: 0.9910
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [6]:
m='remove_Charge'
a=['AtomPairFP','Connectivity',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 332ms/step - loss: 2.0871 - auc: 0.4621
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.5718 - auc: 0.8040
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6486 - auc: 0.8428
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6739 - auc: 0.8573
Epoch 5/100
1/1 [==============================] - 0s 41ms/step - loss: 1.6212 - auc: 0.8657
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4736 - auc: 0.8837
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3136 - auc: 0.8965
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2028 - auc: 0.9018
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1211 - auc: 0.9073
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1001 - auc: 0.8962
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2299 - auc: 0.9906
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2268 - auc: 0.9916
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2304 - auc: 0.9910
Epoch 91/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2346 - auc: 0.9907
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2433 - auc: 0.9893
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2203 - auc: 0.9931
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2334 - auc: 0.9899
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2342 - auc: 0.9903
Epoch 96/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2356 - auc: 0.9902
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2186 - auc: 0.9925
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [7]:
m='remove_Kappa'
a=['AtomPairFP','Connectivity',
       'Charge','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 1.9542 - auc: 0.5034
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6004 - auc: 0.8004
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6516 - auc: 0.8394
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6328 - auc: 0.8519
Epoch 5/100
1/1 [==============================] - 0s 38ms/step - loss: 1.5183 - auc: 0.8656
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3603 - auc: 0.8784
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2437 - auc: 0.8830
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1350 - auc: 0.8954
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1314 - auc: 0.8779
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0902 - auc: 0.8833
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - l

1/1 [==============================] - 0s 39ms/step - loss: 0.2343 - auc: 0.9909
Epoch 89/100
1/1 [==============================] - 0s 42ms/step - loss: 0.2203 - auc: 0.9935
Epoch 90/100
1/1 [==============================] - 0s 39ms/step - loss: 0.2236 - auc: 0.9926
Epoch 91/100
1/1 [==============================] - 0s 40ms/step - loss: 0.2268 - auc: 0.9918
Epoch 92/100
1/1 [==============================] - 0s 44ms/step - loss: 0.2230 - auc: 0.9927
Epoch 93/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2305 - auc: 0.9910
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2382 - auc: 0.9901
Epoch 95/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2222 - auc: 0.9920
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2282 - auc: 0.9911
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2297 - auc: 0.9907
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [8]:
m='remove_MAP4'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 410ms/step - loss: 3.0066 - auc: 0.2485
Epoch 2/100
1/1 [==============================] - 0s 39ms/step - loss: 1.7787 - auc: 0.6143
Epoch 3/100
1/1 [==============================] - 0s 47ms/step - loss: 1.5497 - auc: 0.8231
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6495 - auc: 0.8506
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7905 - auc: 0.8552
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8355 - auc: 0.8552
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8079 - auc: 0.8591
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7172 - auc: 0.8706
Epoch 9/100
1/1 [==============================] - 0s 38ms/step - loss: 1.5933 - auc: 0.8785
Epoch 10/100
1/1 [==============================] - 0s 47ms/step - loss: 1.4299 - auc: 0.8841
Epoch 11/100
1/1 [==============================] - 0s 53ms/step - l

1/1 [==============================] - 0s 48ms/step - loss: 0.2959 - auc: 0.9840
Epoch 89/100
1/1 [==============================] - 0s 55ms/step - loss: 0.2979 - auc: 0.9830
Epoch 90/100
1/1 [==============================] - 0s 37ms/step - loss: 0.2859 - auc: 0.9862
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2921 - auc: 0.9841
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.3046 - auc: 0.9819
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2865 - auc: 0.9848
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2910 - auc: 0.9843
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2881 - auc: 0.9847
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2909 - auc: 0.9845
Epoch 97/100
1/1 [==============================] - 0s 40ms/step - loss: 0.2792 - auc: 0.9869
Epoch 98/100
1/1 [==============================] - 0s 45ms/step - loss: 

In [9]:
m='remove_MOE'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.9212 - auc: 0.5171
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5565 - auc: 0.8192
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6355 - auc: 0.8371
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6231 - auc: 0.8496
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5034 - auc: 0.8582
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3355 - auc: 0.8715
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2104 - auc: 0.8813
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1372 - auc: 0.8885
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1535 - auc: 0.8593
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1099 - auc: 0.8685
Epoch 11/100
1/1 [==============================] - 0s 36ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2466 - auc: 0.9901
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2583 - auc: 0.9873
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2515 - auc: 0.9892
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2574 - auc: 0.9876
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2434 - auc: 0.9907
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2447 - auc: 0.9905
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2498 - auc: 0.9889
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2522 - auc: 0.9888
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2477 - auc: 0.9896
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2456 - auc: 0.9902
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [10]:
m='remove_MorganFP'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 333ms/step - loss: 1.8828 - auc: 0.7847
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6149 - auc: 0.7604
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6047 - auc: 0.6856
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4546 - auc: 0.7613
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3352 - auc: 0.8334
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2933 - auc: 0.8611
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2104 - auc: 0.8838
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1272 - auc: 0.8984
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 1.0466 - auc: 0.9093
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9762 - auc: 0.9211
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2581 - auc: 0.9879
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2511 - auc: 0.9902
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2534 - auc: 0.9892
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2471 - auc: 0.9905
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2480 - auc: 0.9904
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2492 - auc: 0.9903
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2492 - auc: 0.9901
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2521 - auc: 0.9896
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2494 - auc: 0.9900
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2508 - auc: 0.9899
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [11]:
m='remove_Topology'
a=['AtomPairFP','Connectivity',
       'Charge','Kappa','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 1s 518ms/step - loss: 2.0885 - auc: 0.4308
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5663 - auc: 0.7983
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6352 - auc: 0.8373
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6590 - auc: 0.8541
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5930 - auc: 0.8649
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4638 - auc: 0.8769
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3164 - auc: 0.8862
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1712 - auc: 0.8984
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1305 - auc: 0.8906
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0761 - auc: 0.8965
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2405 - auc: 0.9897
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2323 - auc: 0.9915
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2304 - auc: 0.9920
Epoch 91/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2367 - auc: 0.9909
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2311 - auc: 0.9913
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2315 - auc: 0.9912
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2322 - auc: 0.9912
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2268 - auc: 0.9927
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2281 - auc: 0.9916
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2353 - auc: 0.9900
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [12]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('8feature_remove_one.csv',index=False)
print(de)

                 model       auc
0            all_model  0.949406
1    remove_AtomPairFP  0.933202
2  remove_Connectivity  0.951759
3        remove_Charge  0.948347
4         remove_Kappa  0.950517
5          remove_MAP4  0.945145
6           remove_MOE  0.951477
7      remove_MorganFP  0.950496
8      remove_Topology  0.950418
